<a href="https://colab.research.google.com/github/pulkit93/virtual-insurace-agent/blob/main/virtual_insurance_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-experimental langchain-community langchain-openai openai chromadb pypdf sentence_transformers gradio langchain-together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [11]:
import os
from google.colab import userdata
os.environ["TOGETHER_API_KEY"] = userdata.get('TOGETHER_API_KEY') # user your together ai api key here
# go to https://docs.together.ai/docs/quickstart to register

#document loader
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Doc splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter

# vector store
from langchain_community.vectorstores import Chroma

#llm
from langchain_together import Together

In [9]:
loader = PyPDFDirectoryLoader("Insurance Exam Documents/")
docs = loader.load()

In [10]:
len(docs)

655

In [12]:
docs[16]

Document(metadata={'source': 'Insurance Exam Documents/SN_P1_eng_2021.pdf', 'page': 16}, page_content=' \n2/3   3 Unenforceable  contracts : This means what it says, an unenforceable \ncontract cannot be enforced (or sued on) in a court of law.  However, this is \nnot because it is void, but because some required action has not been taken \n(e.g. stamp duty not paid on a lease of land, marine  insur ance policy not \nissued, etc.).  This defect can be remedied by carrying out the required \naction, so that the contract becomes enforceable (e.g. by issuing a marine \npolicy even after a loss has realised).  \n \nTurning to the elements of contract  themselves, legal textbooks are not \nalways agreed just exactly how many elements should be  found with any one valid \ncontract, but we shall consider six criteria here, as follows (remembering that we \nneed not go into great detail):  \n \n(a) Offer : if no offer is made, obviously there can be no agreement between the \ntwo or more parti

In [13]:
def split_docs(documents, chunk_size=500, chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

In [14]:
pages = split_docs(docs)
len(pages)

3337

In [16]:
pages[500].page_content

'- in relation to a sole proprietorship,  as an individual who ultimately \nowns or controls the carrying on of regulated activities by the sole \nproprietorship;  \n \n- in relation to a partnership, as an individual  who is entitled to or \ncontrols, directly or  indirectly, not less than 15% of the capital or \nprofits of the partnership; is, directly or indirectly, entitled to exercise \nor control the exercise of not less than 15% of the voting rights in the'

In [18]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(pages, embedding_function)

In [21]:
query = "what is risk management?"
response = db.similarity_search(query)
print(response[0])

Document(metadata={'page': 10, 'source': 'Insurance Exam Documents/SN_P1_eng_2021.pdf'}, page_content='(c) as a separate field of knowledge and research, risk management may be said \nto be that branch of management which seeks to:  \n \n  (i) identify ; \n \n(ii) quantify ; and  \n \n(iii) deal with  risks ( whether pure  or speculative ) that threaten an \norgani sation.  Tools or measures of risk handling include:  \n \n- risk avoidance : elimination of the chance of loss of a certain kind \nby not exposing oneself to the peril (e.g. abandoning a nuclear')

In [24]:
response = db.similarity_search_with_relevance_scores(query)
response

[(Document(metadata={'page': 10, 'source': 'Insurance Exam Documents/SN_P1_eng_2021.pdf'}, page_content='(c) as a separate field of knowledge and research, risk management may be said \nto be that branch of management which seeks to:  \n \n  (i) identify ; \n \n(ii) quantify ; and  \n \n(iii) deal with  risks ( whether pure  or speculative ) that threaten an \norgani sation.  Tools or measures of risk handling include:  \n \n- risk avoidance : elimination of the chance of loss of a certain kind \nby not exposing oneself to the peril (e.g. abandoning a nuclear'),
  0.7397256984296732),
 (Document(metadata={'page': 9, 'source': 'Insurance Exam Documents/SN_P1_eng_2021.pdf'}, page_content='involved.  \n \nThe majority of the risks which are insured by commercial insurers are \nparticular risks. Fundamental risks are not nor mally insurable because it is \nconsidered financially infeasible for insurers to handle them commercially.  \n \n \n 1.1.3  Risk Management  \n \n  ‘Risk management’ 

# Build RAG using above

In [25]:
llm = Together(
    model="meta-llama/Llama-2-70b-chat-hf",
    max_tokens=256,
    temperature=0.1,
    top_k=1
)

## similarity score helps to fetch only relevant results

In [26]:
retriever = db.as_retriever(similarity_score_threshold = 0.6)

In [33]:
from langchain.prompts import PromptTemplate
prompt_template = """
    You are a helpful insurance agent. Answer insurance related question using the given context.
    Context: {context}
    User Question: {question}
    Answer in less than 100 words
    If you don't have any context and are unsure of the answer, reply that you don't know about this topic.
    """

PROMPT = PromptTemplate(template = f"[INST] {prompt_template} [/INST]", input_variables=["context", "question"])

In [34]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type='stuff',
    retriever= retriever,
    input_key = 'query',
    return_source_documents = True,
    chain_type_kwargs={"prompt":PROMPT},
    verbose=True

)

In [30]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [35]:
query = "what is indemnity?"
response = chain(query)
to_markdown(response['result'])



> Entering new RetrievalQA chain...

> Finished chain.


>   Indemnity means an exact financial compensation for an insured loss, no more no less. It cannot apply to all types of insurance, such as Life Insurance and Personal Accident Insurance, as the loss cannot be measured precisely in financial terms. In these cases, the insurer may provide compensation through a cash payment, repair, replacement, or reinstatement, at their option.

In [37]:
query = "can minors have a life insurance policy?"
response = chain(query)
to_markdown(response['result'])



> Entering new RetrievalQA chain...

> Finished chain.


>   Yes, minors can have a life insurance policy. According to Section 64A of the Insurance Ordinance, a parent or guardian of a minor is given an insurable interest in that young person. This means that apart from one's spouse, only the relationships mentioned in the ordinance constitute an insurable interest arising from blood or family connection.

In [41]:
query = "what kind of insurance policy is recommended for senior citizens?"
response = chain(query)
print(response)
to_markdown(response['result'])




> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'what kind of insurance policy is recommended for senior citizens?', 'result': "  I can't answer this question without more context. The query requires more specific information, such as the senior citizen's health status, financial situation, and insurance goals. I recommend consulting a licensed insurance professional who can assess the individual's circumstances and offer personalized guidance on the best insurance policy for senior citizens.", 'source_documents': [Document(metadata={'page': 51, 'source': 'Insurance Exam Documents/SN_P1_eng_2021.pdf'}, page_content='4.6 POLICY ADMINISTRATION  \n \n This is another departmental description that may involve overlap with other \nsections or departments mentioned above or below.  The general areas of concern here \nmay be:  \n \n(a) General or Life insurance?  : this is a most important question, since the policy \ndocument  with each has a very different significance.  

>   I can't answer this question without more context. The query requires more specific information, such as the senior citizen's health status, financial situation, and insurance goals. I recommend consulting a licensed insurance professional who can assess the individual's circumstances and offer personalized guidance on the best insurance policy for senior citizens.

In [43]:
query = "under what circumstances can a person withdraw money from their MPF account?"
response = chain(query)
to_markdown(response['result'])



> Entering new RetrievalQA chain...

> Finished chain.


>   According to the given context, a person can withdraw money from their MPF account under the following circumstances:
> 
> 1. Retirement: MPF benefits can be withdrawn when a person retires.
> 2. Death: MPF benefits can be withdrawn by the personal representative of a deceased scheme member.
> 3. Small balance: If the total accrued benefits in an MPF scheme do not exceed $5,000 and the member does not have MPF benefits in any other MPF scheme, they can apply for early withdrawal.
> 4. Other grounds: Claims for payment of MPF benefits on other grounds can be made by the scheme member or a committee of the estate appointed under Mental Health Ordinance (Cap 136).

In [45]:
query = "can someone withdraw money from MPF if they are permanently leaving Hong Kong?"
response = chain(query)
to_markdown(response['result'])



> Entering new RetrievalQA chain...

> Finished chain.


>   Yes, it is possible to withdraw money from MPF if you are permanently leaving Hong Kong. According to the regulations, you can withdraw the "minimum MPF benefits" upon permanent departure from Hong Kong. However, this reason for early withdrawal can only be used once in a lifetime. It's important to note that there may be additional requirements or restrictions imposed by the specific MPF scheme, so it's best to consult the scheme's governing rules or a qualified professional for more information.